In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [2]:
conf = SparkConf()
ss = SparkSession(sc)
ss.udf.registerJavaFunction("parseUrl", "com.github.silentsokolov.App", t.ArrayType(t.StringType(), True))

In [3]:
df = ss.read.csv(['/path/to/data'], sep='\t')
df = df.select(
    f.col('_c4')
).limit(1000000).cache()
df.show()

+--------------------+
|                 _c4|
+--------------------+
|https://yastatic....|
|https://filmskino...|
|https://avr.tiviz...|
|https://xn--80aaj...|
|https://bcp.crwdc...|
|https://profter.r...|
|https://filmskino...|
|https://moevideo....|
|http://f1analytic...|
|https://valse-bos...|
|https://profile.s...|
|https://tag.rutar...|
|https://moevideo....|
|https://playrepla...|
|https://avr.i-tra...|
|https://filmskino...|
|https://www.speed...|
|https://kinomans....|
|https://a.many-fi...|
|https://playrepla...|
+--------------------+
only showing top 20 rows



In [4]:
from urllib.parse import urlparse

@f.udf(t.ArrayType(t.StringType(), True))
def parse_py(v):
    try:
        o = urlparse(v)
        return [o.scheme, o.netloc.replace('www.', '', 1), o.path + o.query + o.fragment]
    except:
        return None

In [5]:
%timeit df.withColumn('v', parse_py(f.col('_c4'))).agg(f.count(f.col('v')[0])).show()

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|    1000000|
+-----------+

14 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

df.select(
    f.expr('parse_url(_c4, "PROTOCOL")').alias('scheme'),
    f.regexp_replace(
        f.expr('parse_url(_c4, "HOST")'),
        r'^(www\.)',
        ''
    ).alias('domain'),
    f.when(
        f.expr('parse_url(_c4, "REF")').isNull(),
        f.expr('parse_url(_c4, "FILE")')
    ).otherwise(
        f.concat(
            f.expr('parse_url(_c4, "FILE")'),
            f.lit('#'),
            f.expr('parse_url(_c4, "REF")')
        )
    ).alias('path'),
).withColumn('v', f.array(f.col('scheme'), f.col('domain'), f.col('path'))).agg(f.count(f.col('v')[0])).show()

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999590|
+-----------+

7 s ± 197 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

df.withColumn('v', f.expr('parseUrl(_c4)')).agg(f.count(f.col('v')[0])).show()

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

+-----------+
|count(v[0])|
+-----------+
|     999998|
+-----------+

7.66 s ± 459 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
